In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

In [22]:
billboard = pd.read_csv('no_match.csv') 
spotify = pd.read_csv('top10000song.csv')

In [23]:
spotify = spotify.rename(columns={'Track Name': 'name', 'Artist Name(s)': 'artists'}) 

print(billboard.columns)
billboard.drop(['valence','danceability','energy','tempo'], axis = 1, inplace=True)
print(billboard.columns) 

Index(['Year', 'Rank', 'Artist', 'Song', 'valence', 'danceability', 'energy',
       'tempo'],
      dtype='object')
Index(['Year', 'Rank', 'Artist', 'Song'], dtype='object')


In [24]:
def remove_accents(text):
    if isinstance(text, str):
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    return text

# Clean and normalize 'Artist' and 'Song' columns in the no_match dataset
billboard['Artist'] = billboard['Artist'].apply(remove_accents) \
                                    .str.lower() \
                                    .str.replace(r'\(.*?\)', '', regex=True) \
                                    .str.replace(r'[^a-z\s]', '', regex=True) \
                                    .str.strip()

billboard['Song'] = billboard['Song'].apply(remove_accents) \
                                  .str.lower() \
                                  .str.replace(r'\(.*?\)', '', regex=True) \
                                  .str.replace(r'[^a-z\s]', '', regex=True) \
                                  .str.strip()

# Clean and normalize 'Artist Name(s)' and 'Track Name' columns in the Spotify dataset
spotify['artists'] = spotify['artists'].apply(remove_accents) \
                                                      .str.lower() \
                                                      .str.replace(r'\(.*?\)', '', regex=True) \
                                                      .str.replace(r'[^a-z\s]', '', regex=True) \
                                                      .str.strip()

spotify['name'] = spotify['name'].apply(remove_accents) \
                                              .str.lower() \
                                              .str.replace(r'\(.*?\)', '', regex=True) \
                                              .str.replace(r'[^a-z\s]', '', regex=True) \
                                              .str.strip()

# Attempt to merge the cleaned datasets on 'Artist' and 'Song'/'Track Name'
merged_df = duckdb.sql("""
                        SELECT billboard.Year, billboard.Rank, billboard.Artist, billboard.Song, 
                               MEDIAN(spotify.valence) AS valence, MEDIAN(spotify.danceability) AS danceability, 
                               MEDIAN(spotify.energy) AS energy, MEDIAN(spotify.tempo) AS tempo
                        FROM billboard
                        LEFT JOIN spotify ON billboard.Song = spotify.name AND billboard.Artist = spotify.artists
                        GROUP BY billboard.Year, billboard.Artist, billboard.Song, billboard.Rank
                        ORDER BY billboard.Year, billboard.Rank
                    """).df()


print(merged_df.shape)
# Checking how many matches were found after cleaning

unmatched_rows = merged_df[merged_df['valence'].isna()]



after =duckdb.sql("""
                       SELECT COUNT(*) 
                       FROM unmatched_rows
                       WHERE Year < 2020 AND Year >= 2010  
                    """).df()
print(after) 


print(unmatched_rows.shape) 


# matched_rows[['Year', 'Rank', 'Artist', 'Song']].head(), unmatched_rows[['Year', 'Rank', 'Artist', 'Song']].head(), unmatched_rows.shape


merged_df.to_csv('second_merge.csv', index = False) 

(2124, 8)
   count_star()
0           122
(1937, 8)
